# Train GRU on PerceptionEncoder Backbone

In [ ]:
!git clone --recursive https://github.com/perso00k/aml-2025-mistake-detection.git code

Cloning into 'code'...
remote: Enumerating objects: 1309, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 1309 (delta 17), reused 71 (delta 13), pack-reused 1224 (from 2)
Receiving objects: 100% (1309/1309), 1.67 GiB | 37.12 MiB/s, done.
Resolving deltas: 100% (307/307), done.
Submodule 'annotations' (https://github.com/CaptainCook4D/annotations) registered for path 'annotations'
Cloning into '/content/code/annotations'...
remote: Enumerating objects: 152, done.        
remote: Counting objects: 100% (152/152), done.        
remote: Compressing objects: 100% (98/98), done.        
remote: Total 152 (delta 75), reused 108 (delta 46), pack-reused 0 (from 0)        
Receiving objects: 100% (152/152), 793.14 KiB | 6.90 MiB/s, done.
Resolving deltas: 100% (75/75), done.
Submodule path 'annotations': checked out '0e9a108be2cbcbcbd592e7418c0ab9c16232d27a'


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import shutil
import os
from tqdm import tqdm

# Percorsi (Verifica che la cartella source contenga la sottocartella features_PE o direttamente i file)
# Se i tuoi npz sono dentro ".../features_PE", punta lì.
source_dir = "/content/drive/MyDrive/AML_Project_resized_ds/features_PE_npz_1s_1s"
dest_dir = "code/data/video/perception_encoder" # Dove li vuole il dataloader

os.makedirs(dest_dir, exist_ok=True)

print(f"Inizio copia da {source_dir} a {dest_dir}...")

# Ottieni lista file .npz (o .pt se non li hai ancora convertiti)
files = [f for f in os.listdir(source_dir) if f.endswith('.npz') or f.endswith('.pt')]

if len(files) == 0:
    print("⚠️ ATTENZIONE: Nessun file .npz o .pt trovato! Controlla il percorso source_dir.")
else:
    for filename in tqdm(files):
        src = os.path.join(source_dir, filename)
        dst = os.path.join(dest_dir, filename)

        # Copia solo se non esiste già (così se rilanci non perde tempo)
        if not os.path.exists(dst):
            shutil.copy2(src, dst)

    print("✅ Copia completata.")

Inizio copia da /content/drive/MyDrive/AML_Project_resized_ds/features_PE_npz_1s_1s a code/data/video/perception_encoder...


100%|██████████| 384/384 [00:40<00:00,  9.48it/s]

✅ Copia completata.


In [ ]:
!pip install torcheval
!pip install loguru

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 4.6 MB/s eta 0:00:00


In [ ]:
import os

# Percorso delle feature
target_dir = "/content/code/data/video/perception_encoder"

print(f"📂 Analisi cartella: {target_dir}")

if not os.path.exists(target_dir):
    print("❌ Errore: La cartella non esiste!")
else:
    count = 0
    files = os.listdir(target_dir)

    for filename in files:
        # Cerchiamo il pezzo di stringa che "rompe le scatole"
        if "_224_360p" in filename:
            # Lo rimuoviamo sostituendolo con una stringa vuota
            # Es: 25_109_360p_224_360p.mp4_1s_1s.npz -> 25_109_360p.mp4_1s_1s.npz
            new_filename = filename.replace("_224_360p", "")

            src = os.path.join(target_dir, filename)
            dst = os.path.join(target_dir, new_filename)

            # Eseguiamo la rinomina
            os.rename(src, dst)
            # print(f"✅ Rinominato: {filename} -> {new_filename}") # Decommenta se vuoi vedere la lista
            count += 1

    if count == 0:
        print("⚠️ Nessun file trovato con il pattern '_224_360p'. Sono già a posto?")
    else:
        print(f"🎉 Fatto! Ho corretto {count} file.")

📂 Analisi cartella: /content/code/data/video/perception_encoder
🎉 Fatto! Ho corretto 384 file.


In [ ]:
%cd code

/content/code


In [ ]:
!python train_er.py \
  --backbone perception_encoder \
  --variant BiGRU \
  --batch_size 8 \
  --lr 5e-4 \
  --weight_decay 1e-4 \
  --num_epochs 40 \
  --ckpt_directory checkpoints_test/bigru_pe_step_std \
  --split step \
  --modality video

wandb: Currently logged in as: bennycutrone19 (bennycutrone19-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.23.1
wandb: Run data is saved locally in /content/code/wandb/run-20260111_090902-k11nux2a
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run happy-plasma-4
wandb: ⭐️ View project at https://wandb.ai/bennycutrone19-politecnico-di-torino/error_recognition_step_perception_encoder_BiGRU_video
wandb: 🚀 View run at https://wandb.ai/bennycutrone19-politecnico-di-torino/error_recognition_step_perception_encoder_BiGRU_video/runs/k11nux2a
-------------------------------------------------------------
Training step model and testing on step level
Train args: {'num_workers': 0, 'pin_memory': False, 'shuffle': True, 'batch_size': 8}
Test args: {'num_workers': 0, 'pin_memory': False, 'shuffle': False, 'batch_size': 1}
{

In [ ]:
!python -m core.evaluate --variant BiGRU --backbone perception_encoder --ckpt /content/code/checkpoints_test/bigru_pe_step_std/error_recognition/BiGRU/perception_encoder/error_recognition_step_perception_encoder_BiGRU_video_epoch_17.pt --split step --threshold 0.6

Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
test Progress: 42347/798: 100% 798/798 [00:25<00:00, 30.93it/s]
----------------------------------------------------------------
test Sub Step Level Metrics: {'precision': 0.387945635559962, 'recall': 0.7922220347562005, 'f1': 0.5208396883059261, 'accuracy': 0.5919663730606655, 'auc': np.float64(0.7328042706280737), 'pr_auc': tensor(0.3655)}
test Step Level Metrics: {'precision': 0.5155807365439093, 'recall': 0.7309236947791165, 'f1': 0.6046511627906976, 'accuracy': 0.7017543859649122, 'auc': np.float64(0.7544933833695437), 'pr_auc': tensor(0.4608)}
----------------------------------------------------------------


#### Tentavi extra

In [ ]:
!python train_er.py \
  --backbone perception_encoder \
  --variant BiGRU \
  --batch_size 8 \
  --lr 5e-4 \
  --weight_decay 1e-3 \
  --num_epochs 40 \
  --ckpt_directory checkpoints_test/bigru_pe_step_mid_reg \
  --split step \
  --modality video

wandb: Currently logged in as: bennycutrone19 (bennycutrone19-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.23.1
wandb: Run data is saved locally in /content/code/wandb/run-20260111_114025-vulo9cig
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run flowing-cherry-5
wandb: ⭐️ View project at https://wandb.ai/bennycutrone19-politecnico-di-torino/error_recognition_step_perception_encoder_BiGRU_video
wandb: 🚀 View run at https://wandb.ai/bennycutrone19-politecnico-di-torino/error_recognition_step_perception_encoder_BiGRU_video/runs/vulo9cig
-------------------------------------------------------------
Training step model and testing on step level
Train args: {'num_workers': 0, 'pin_memory': False, 'shuffle': True, 'batch_size': 8}
Test args: {'num_workers': 0, 'pin_memory': False, 'shuffle': False, 'batch_size': 1}

In [ ]:
!python train_er.py \
  --backbone perception_encoder \
  --variant BiGRU \
  --batch_size 8 \
  --lr 1e-4 \
  --weight_decay 1e-4 \
  --num_epochs 50 \
  --ckpt_directory checkpoints_test/bigru_pe_step_low_lr \
  --split step \
  --modality video

wandb: Currently logged in as: bennycutrone19 (bennycutrone19-politecnico-di-torino) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: ⢿ Waiting for wandb.init()...
wandb: ⣻ Waiting for wandb.init()...
wandb: ⣽ Waiting for wandb.init()...
wandb: Tracking run with wandb version 0.23.1
wandb: Run data is saved locally in /content/code/wandb/run-20260111_134934-1ab5n61h
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run worldly-microwave-6
wandb: ⭐️ View project at https://wandb.ai/bennycutrone19-politecnico-di-torino/error_recognition_step_perception_encoder_BiGRU_video
wandb: 🚀 View run at https://wandb.ai/bennycutrone19-politecnico-di-torino/error_recognition_step_perception_encoder_BiGRU_video/runs/1ab5n61h
-------------------------------------------------------------
Training step model and testing on step level
Train args: {'num_workers': 0, 'pin_memory': False, 'shuffle': True, 'batch_size': 8}
Test args: {'num_workers': 0, 'pin_memory': F

In [ ]:
!python -m core.evaluate --variant BiGRU --backbone perception_encoder --ckpt /content/code/checkpoints_test/bigru_pe_step_low_lr/error_recognition/BiGRU/perception_encoder/error_recognition_step_perception_encoder_BiGRU_video_epoch_13.pt --split step --threshold 0.65

Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
test Progress: 42347/798: 100% 798/798 [00:26<00:00, 30.23it/s]
----------------------------------------------------------------
test Sub Step Level Metrics: {'precision': 0.36057587435468624, 'recall': 0.8366796018221697, 'f1': 0.5039634146341463, 'accuracy': 0.5389519918766382, 'auc': np.float64(0.7190170713587645), 'pr_auc': tensor(0.3474)}
test Step Level Metrics: {'precision': 0.5121951219512195, 'recall': 0.6746987951807228, 'f1': 0.5823223570190641, 'accuracy': 0.6979949874686717, 'auc': np.float64(0.741669775641729), 'pr_auc': tensor(0.4471)}
----------------------------------------------------------------


# Evaluation su recordings

In [ ]:
!python -m core.evaluate --variant BiGRU --backbone perception_encoder --ckpt /content/code/checkpoints_test/bigru_pe_step_std/error_recognition/BiGRU/perception_encoder/error_recognition_step_perception_encoder_BiGRU_video_epoch_17.pt --split recordings --threshold 0.5

Loaded annotations...... 
Loading recording ids from recordings_combined_splits.json
test Progress: 38340/671: 100% 671/671 [00:23<00:00, 29.14it/s]
----------------------------------------------------------------
test Sub Step Level Metrics: {'precision': 0.46950522893048596, 'recall': 0.8475450146743873, 'f1': 0.6042697582355436, 'accuracy': 0.6349765258215962, 'auc': np.float64(0.7797461694321047), 'pr_auc': tensor(0.4481)}
test Step Level Metrics: {'precision': 0.4883177570093458, 'recall': 0.8672199170124482, 'f1': 0.624813153961136, 'accuracy': 0.6259314456035767, 'auc': np.float64(0.759866833928399), 'pr_auc': tensor(0.4712)}
----------------------------------------------------------------
